<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#version 330 core

in vec2 TexCoord; // 텍스처 좌표
out vec4 FragColor; // 프래그먼트 색상

uniform sampler2D textureSampler; // 텍스처 유니폼

void main()
{
    vec4 texColor = texture(textureSampler, TexCoord); // 텍스처 샘플링

    // 빨간색과 파란색의 합이 마젠타 범위에 있으면 마젠타로 설정
    if (texColor.r + texColor.b > 0.8 && texColor.b > 0.2)
    {
        FragColor = vec4(1.0, 0.0, 1.0, 1.0); // 마젠타 색상 설정
    }
    else
    {
        discard; //  무시
    }
}

네, 맞습니다. 텍스처 유닛이 3개인 경우에는 각각의 텍스처 유닛에 대한 샘플러 유니폼 변수를 선언하고 사용할 수 있습니다.

예를 들어, 텍스처 유닛 0, 1, 2에 대한 샘플러 유니폼 변수를 만들어 사용할 수 있습니다. 아래는 간단한 예제 코드입니다:

```glsl
#version 330 core

in vec2 TexCoord; // 텍스처 좌표
out vec4 FragColor; // 프래그먼트 색상

uniform sampler2D textureSampler0; // 텍스처 유니폼 0
uniform sampler2D textureSampler1; // 텍스처 유니폼 1
uniform sampler2D textureSampler2; // 텍스처 유니폼 2

void main()
{
    // 텍스처 유닛 0에서 텍스처 샘플링
    vec4 texColor0 = texture(textureSampler0, TexCoord);
    
    // 텍스처 유닛 1에서 텍스처 샘플링
    vec4 texColor1 = texture(textureSampler1, TexCoord);
    
    // 텍스처 유닛 2에서 텍스처 샘플링
    vec4 texColor2 = texture(textureSampler2, TexCoord);

    // 텍스처 샘플링 결과를 조합하여 원하는 처리 수행
    FragColor = texColor0 * texColor1 * texColor2;
}
```

위 코드에서 `textureSampler0`, `textureSampler1`, `textureSampler2`는 각각 텍스처 유닛 0, 1, 2에 대한 샘플러 유니폼 변수를 나타냅니다. 이렇게 선언된 샘플러 유니폼 변수를 사용하여 각 텍스처 유닛에서 텍스처를 샘플링할 수 있습니다.


외부에서 특정 텍스처 유닛을 지정하려면 GLSL 쉐이더 프로그램을 사용하는 동안 해당 텍스처 유닛에 대한 유니폼 변수를 설정하면 됩니다. 이렇게 하면 쉐이더가 실행되는 동안에도 특정 텍스처 유닛을 사용할 수 있습니다.

아래는 간단한 예제 코드로 설명하겠습니다:

```cpp
// OpenGL에서 shader program을 사용하기 전에 해당 program을 활성화해야 함
glUseProgram(shaderProgramID);

// 특정 텍스처 유닛을 지정할 때 유니폼 변수에 값을 설정
int textureUnit = 2; // 예: 텍스처 유닛 2를 사용하고 싶을 때
glUniform1i(glGetUniformLocation(shaderProgramID, "textureSampler"), textureUnit);
```

위 코드에서 `glGetUniformLocation` 함수를 사용하여 쉐이더 프로그램 내에서 `textureSampler`라는 샘플러 유니폼 변수의 위치를 얻은 후에 `glUniform1i` 함수를 사용하여 해당 위치에 원하는 텍스처 유닛 값을 설정합니다. 이렇게 하면 쉐이더가 실행될 때 지정된 텍스처 유닛을 사용하여 텍스처를 샘플링하게 됩니다.

따라서 외부에서 특정 텍스처 유닛을 지정하려면 해당 유니폼 변수에 값을 설정하는 방식을 사용하면 됩니다.

!!ARBfp1.0
TEMP texColor; // 텍스처 샘플링 결과를 저장할 변수

TEX texColor, fragment.texcoord[0], texture[0], 2D; // 텍스처 샘플링

// 빨간색과 파란색의 합이 마젠타 범위에 있으면 마젠타로 설정
ADD_SAT texColor.rgb, texColor.r, texColor.b;
SGE_SAT texColor.a, texColor.b, 0.2;
SGE_SAT texColor.a, texColor.a, 0.8;
MUL texColor.a, texColor.a, texColor.a; // a 값을 계산

SUB_SAT result.color.rgb, {1, 1, 1}, texColor.a; // 마젠타 색상 설정
MOV result.color.a, 1.0; // 알파 값 설정
END

!!ARBfp1.0

PARAM constantColor = {1.0, 0.0, 1.0, 1.0}; // 마젠타 색상
TEMP texColor;

# 텍스처 좌표와 텍스처 유니폼을 가져옵니다.
TEX texColor, fragment.texcoord, texture[0], 2D;

# 빨간색과 파란색의 합이 마젠타 범위에 있으면 마젠타로 설정합니다.
ADD_SAT result.color, texColor.r, texColor.b;
CMP result.color, -result.color, constantColor, texColor;

# 파란색이 0.2보다 크면 결과를 유지합니다.
SGE result.color.b, texColor.b, 0.2;
MUL result.color, result.color, result.color.b;

END

!!ARBfp1.0

# 텍스처 유닛 선언
PARAM textureUnit = { 0 };

# 텍스처 샘플링을 위한 좌표
TEMP texCoord;
MOV texCoord, fragment.texcoord;

# 텍스처에서 색상 샘플링
TEMP texColor;
TEX texColor, texCoord, texture[textureUnit], 2D;

# 빨간색과 파란색 합 계산
TEMP colorSum;
ADD colorSum.r, texColor.r, texColor.b;

# 조건 확인 및 마젠타색 설정
TEMP magenta;
PARAM magentaColor = { 1.0, 0.0, 1.0, 1.0 };
PARAM threshold = { 0.8, 0.2, 0, 0 };
SGE magenta, colorSum.r, threshold.x; # colorSum.r >= 0.8 이면 magenta = 1
SGE magenta.a, texColor.b, threshold.y; # texColor.b > 0.2 이면 magenta.a = 1
MUL magenta, magenta, magenta.a; # 두 조건 모두 만족 시 magenta = 1
CMP result.color, -magenta, texColor, magentaColor; # magenta == 1이면 마젠타색, 아니면 원본 색상

# 프래그먼트 색상 출력
MOV result.color, magentaColor;

END



CMP result.color, -condition.a, magenta, {0.0, 0.0, 0.0, 0.0};